In [2]:
from pygam import LinearGAM, s, f

In [3]:
import dask.dataframe as ddf
import pandas as pd
import datetime
import functools
import seaborn as sns
import matplotlib.pyplot as plt

### Import suicide death data

In [4]:
years = [year for year in range(2000, 2020)]
years

[2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [5]:
data = []
for year in years:
    suicide_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/Suicide_Death_NCHS/suicide_patient_level_1960_2020/overall/overall_suicide_patient_level_' + str(year) + '.csv', dtype = {'year': int, 'county_residence': str, 'county_death': str, 'state_residence': str, 'state_death': str})
    suicide_year = suicide_year.loc[:, ~suicide_year.columns.str.contains('^Unnamed')]
    data.append(suicide_year)

In [6]:
suicide = pd.concat(data)
suicide.head()

,year,county_residence,state_residence,state_death,county_death,month,sex,race,age,age_range,death_cause
0,2000,01115,01,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide
1,2000,01101,01,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide
2,2000,01001,01,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide
3,2000,01003,01,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide
4,2000,01015,01,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide


In [7]:
suicide = suicide.drop(['county_residence', 'state_residence'], axis = 1)
suicide = suicide.rename(columns = {'state_death': 'statefips', 'county_death': 'fips'})
suicide.head()

,year,statefips,fips,month,sex,race,age,age_range,death_cause
0,2000,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide
1,2000,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide
2,2000,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide
3,2000,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide
4,2000,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide


In [8]:
suicide

,year,statefips,fips,month,sex,race,age,age_range,death_cause
0,2000,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide
1,2000,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide
2,2000,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide
3,2000,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide
4,2000,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide
...,...,...,...,...,...,...,...,...,...
47669,2019,36,36081,Dec,Male,Other,5,age 25-64,nonfirearm_suicide
47670,2019,36,36081,Dec,Male,Black,5,age 25-64,nonfirearm_suicide
47671,2019,36,36005,Dec,Female,Black,4,age 0-24,nonfirearm_suicide
47672,2019,36,36005,Dec,Male,White,6,age 25-64,nonfirearm_suicide


### Add region information

In [9]:
state = pd.read_excel('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/temp_bins_suicide/test_paper/state_code.xlsx', engine='openpyxl', dtype = {'Code': int})
state.head()

,Code,State,Abbreviation,Alpha code
0,1,Alabama,Ala.,AL
1,2,Alaska,NaN,AK
2,4,Arizona,Ariz.,AZ
3,5,Arkansas,Ark.,AR
4,6,California,Calif.,CA


In [10]:
def convertStateCode(code):
    return str(code).rjust(2, '0')

In [11]:
state['Code'] = state['Code'].apply(convertStateCode)
state = state.rename(columns = {'Code': 'statefips', 'State': 'state_name', 'Alpha code': 'state'})
state.head()

,statefips,state_name,Abbreviation,state
0,01,Alabama,Ala.,AL
1,02,Alaska,NaN,AK
2,04,Arizona,Ariz.,AZ
3,05,Arkansas,Ark.,AR
4,06,California,Calif.,CA


In [12]:
suicide['statefips'].unique()

array(['01', 'na', '04', '05', '06', '08', '09', '10', '11', '12', '13',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48',
       '49', '50', '51', '53', '54', '55', '56', '02'], dtype=object)

In [13]:
state

,statefips,state_name,Abbreviation,state
0,01,Alabama,Ala.,AL
1,02,Alaska,NaN,AK
2,04,Arizona,Ariz.,AZ
3,05,Arkansas,Ark.,AR
4,06,California,Calif.,CA
5,08,Colorado,Colo.,CO
6,09,Connecticut,Conn.,CT
7,10,Delaware,Del.,DE
8,11,District of Columbia,D.C.,DC
9,12,Florida,Fla.,FL


In [14]:
state['statefips'].unique()

array(['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48',
       '49', '50', '51', '53', '54', '55', '56'], dtype=object)

In [15]:
suicide = suicide.merge(state[['statefips', 'state_name', 'state']], on = ['statefips'], how = 'inner')
suicide

,year,statefips,fips,month,sex,race,age,age_range,death_cause,state_name,state
0,2000,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide,Alabama,AL
1,2000,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide,Alabama,AL
2,2000,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide,Alabama,AL
3,2000,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide,Alabama,AL
4,2000,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide,Alabama,AL
...,...,...,...,...,...,...,...,...,...,...,...
763871,2019,02,02020,Dec,Male,White,4,age 0-24,firearm_suicide,Alaska,AK
763872,2019,02,02170,Oct,Male,White,8,age 25-64,firearm_suicide,Alaska,AK
763873,2019,02,02050,Dec,Male,White,4,age 0-24,nonfirearm_suicide,Alaska,AK
763874,2019,02,02122,Dec,Male,White,5,age 25-64,firearm_suicide,Alaska,AK


In [56]:
suicide_northeast = suicide[suicide['region'] == 'Northeast']
suicide_northeast['statefips'].unique()

array(['09', '10', '11', '23', '24', '25', '33', '34', '36', '42', '44',
       '50'], dtype=object)

In [16]:
west_states = ['Colorado', 'Wyoming', 'Montana', 'Idaho', 'Washington',
              'Oregon', 'Utah', 'Nevada', 'California', 'Alaska', 'Hawaii']
Midwest_states = ['Ohio', 'Indiana', 'Michigan', 'Illinois', 'Missouri', 
                  'Wisconsin', 'Minnesota', 'Iowa', 'Kansas', 'Nebraska', 'South Dakota', 'North Dakota']
Southwest_states = ['Texas', 'Oklahoma', 'New Mexico', 'Arizona']
Southeast_states = ['West Virginia', 'Virginia', 'Kentucky', 'Tennessee', 'North Carolina', 
                    'South Carolina', 'Georgia', 'Alabama', 'Mississippi', 'Arkansas', 'Louisiana', 'Florida']
Northeast_states = ['Maine', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New Hampshire', 'Vermont', 
                    'New York', 'Pennsylvania', 'New Jersey', 'Delaware', 'Maryland']

In [17]:
#use a function to get region information
def get_region(state):
    if state in west_states:
        return 'West'
    elif state in Midwest_states:
        return 'Midwest'
    elif state in Southwest_states:
        return 'Southwest'
    elif state in Southeast_states:
        return 'Southeast'
    else:
        return 'Northeast'

In [18]:
suicide['region'] = suicide['state_name'].apply(get_region)
suicide

,year,statefips,fips,month,sex,race,age,age_range,death_cause,state_name,state,region
0,2000,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide,Alabama,AL,Southeast
1,2000,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide,Alabama,AL,Southeast
2,2000,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide,Alabama,AL,Southeast
3,2000,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide,Alabama,AL,Southeast
4,2000,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide,Alabama,AL,Southeast
...,...,...,...,...,...,...,...,...,...,...,...,...
763871,2019,02,02020,Dec,Male,White,4,age 0-24,firearm_suicide,Alaska,AK,West
763872,2019,02,02170,Oct,Male,White,8,age 25-64,firearm_suicide,Alaska,AK,West
763873,2019,02,02050,Dec,Male,White,4,age 0-24,nonfirearm_suicide,Alaska,AK,West
763874,2019,02,02122,Dec,Male,White,5,age 25-64,firearm_suicide,Alaska,AK,West


### Load population data and add region information

In [19]:
data_pop = []
for year in years:
    pop_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/temp_bins_suicide/Population/population_monthly/population_monthly_' + str(year) + '.csv', dtype = {'year': int, 'fips': str})
    pop_year = pop_year.loc[:, ~pop_year.columns.str.contains('^Unnamed')]
    data_pop.append(pop_year)

In [20]:
pop = pd.concat(data_pop)
pop.head()

,year,fips,pop,month
0,2000,01001,39646,Jan
1,2000,01001,39758,Feb
2,2000,01001,39870,Mar
3,2000,01001,39982,Apr
4,2000,01001,40094,May


In [21]:
def findState(code):
    return code[:2]

In [22]:
pop['statefips'] = pop['fips'].apply(findState)
pop.head()

,year,fips,pop,month,statefips
0,2000,01001,39646,Jan,01
1,2000,01001,39758,Feb,01
2,2000,01001,39870,Mar,01
3,2000,01001,39982,Apr,01
4,2000,01001,40094,May,01


In [23]:
pop = pop.merge(state[['statefips', 'state_name', 'state']], on = ['statefips'], how = 'inner')
pop.head()

,year,fips,pop,month,statefips,state_name,state
0,2000,01001,39646,Jan,01,Alabama,AL
1,2000,01001,39758,Feb,01,Alabama,AL
2,2000,01001,39870,Mar,01,Alabama,AL
3,2000,01001,39982,Apr,01,Alabama,AL
4,2000,01001,40094,May,01,Alabama,AL


In [24]:
pop['region'] = pop['state_name'].apply(get_region)
pop.head()

,year,fips,pop,month,statefips,state_name,state,region
0,2000,01001,39646,Jan,01,Alabama,AL,Southeast
1,2000,01001,39758,Feb,01,Alabama,AL,Southeast
2,2000,01001,39870,Mar,01,Alabama,AL,Southeast
3,2000,01001,39982,Apr,01,Alabama,AL,Southeast
4,2000,01001,40094,May,01,Alabama,AL,Southeast


In [25]:
month_replace = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
                'Nov': 11, 'Dec': 12}
pop['month'] = pop['month'].replace(month_replace)
pop['month'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [26]:
pop.head()

,year,fips,pop,month,statefips,state_name,state,region
0,2000,01001,39646,1,01,Alabama,AL,Southeast
1,2000,01001,39758,2,01,Alabama,AL,Southeast
2,2000,01001,39870,3,01,Alabama,AL,Southeast
3,2000,01001,39982,4,01,Alabama,AL,Southeast
4,2000,01001,40094,5,01,Alabama,AL,Southeast


In [27]:
pop_month = pop.drop(['fips', 'statefips', 'state_name', 'state'], axis = 1)
pop_month = pop_month.groupby(['year', 'region', 'month']).sum()
pop_month = pop_month.reset_index()
pop_month.head()

,year,region,month,pop
0,2000,Midwest,1,58569836
1,2000,Midwest,2,58583129
2,2000,Midwest,3,58596410
3,2000,Midwest,4,58609705
4,2000,Midwest,5,58622795


In [28]:
pop_month

,year,region,month,pop
0,2000,Midwest,1,58569836
1,2000,Midwest,2,58583129
2,2000,Midwest,3,58596410
3,2000,Midwest,4,58609705
4,2000,Midwest,5,58622795
...,...,...,...,...
1195,2019,West,8,62659537
1196,2019,West,9,62720439
1197,2019,West,10,62781343
1198,2019,West,11,62842243


In [29]:
suicide

,year,statefips,fips,month,sex,race,age,age_range,death_cause,state_name,state,region
0,2000,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide,Alabama,AL,Southeast
1,2000,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide,Alabama,AL,Southeast
2,2000,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide,Alabama,AL,Southeast
3,2000,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide,Alabama,AL,Southeast
4,2000,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide,Alabama,AL,Southeast
...,...,...,...,...,...,...,...,...,...,...,...,...
763871,2019,02,02020,Dec,Male,White,4,age 0-24,firearm_suicide,Alaska,AK,West
763872,2019,02,02170,Oct,Male,White,8,age 25-64,firearm_suicide,Alaska,AK,West
763873,2019,02,02050,Dec,Male,White,4,age 0-24,nonfirearm_suicide,Alaska,AK,West
763874,2019,02,02122,Dec,Male,White,5,age 25-64,firearm_suicide,Alaska,AK,West


In [30]:
suicide_death = suicide.drop(['statefips', 'fips', 'sex', 'race', 'age', 'age_range', 'state_name', 'state'], axis = 1)
suicide_death = suicide_death.groupby(['year', 'region', 'month']).count()
suicide_death = suicide_death.reset_index()
suicide_death.head()

,year,region,month,death_cause
0,2000,Midwest,Apr,536
1,2000,Midwest,Aug,583
2,2000,Midwest,Dec,471
3,2000,Midwest,Feb,498
4,2000,Midwest,Jan,584


In [31]:
suicide_death['year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [32]:
suicide_death['month'] = suicide_death['month'].replace(month_replace)
suicide_death.head()

,year,region,month,death_cause
0,2000,Midwest,4,536
1,2000,Midwest,8,583
2,2000,Midwest,12,471
3,2000,Midwest,2,498
4,2000,Midwest,1,584


In [33]:
suicide_pop = suicide_death.merge(pop_month, on = ['year', 'month', 'region'], how = 'inner')
suicide_pop.head()

,year,region,month,death_cause,pop
0,2000,Midwest,4,536,58609705
1,2000,Midwest,8,583,58662683
2,2000,Midwest,12,471,58715987
3,2000,Midwest,2,498,58583129
4,2000,Midwest,1,584,58569836


In [34]:
suicide_pop = suicide_pop.rename(columns = {'death_cause': 'deaths'})
suicide_pop['suicide_rate'] = (suicide_pop['deaths']/suicide_pop['pop'])*100000
suicide_pop.head()

,year,region,month,deaths,pop,suicide_rate
0,2000,Midwest,4,536,58609705,0.914524
1,2000,Midwest,8,583,58662683,0.993817
2,2000,Midwest,12,471,58715987,0.802167
3,2000,Midwest,2,498,58583129,0.850074
4,2000,Midwest,1,584,58569836,0.997100


### Load temperature and precipitation data from PRISM

In [35]:
data_temp = []
for year in years:
    temp_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/PRISM_Data/PRISM_daily_county_level/prism_daily_county_level_' + str(year) + '.csv', dtype = {'year': int, 'fips': str})
    temp_year = temp_year.loc[:, ~temp_year.columns.str.contains('^Unnamed')]
    data_temp.append(temp_year)

In [36]:
climate = pd.concat(data_temp)
climate.head()

,fips,date,tMean,tMin,tMax,prec,year
0,01001,2000-01-01,16.262459,10.089380,22.435537,0.560423,2000
1,01001,2000-01-02,17.412651,11.954220,22.871081,0.390251,2000
2,01001,2000-01-03,18.705264,14.001039,23.409490,0.216170,2000
3,01001,2000-01-04,16.203456,9.861904,22.545009,16.488676,2000
4,01001,2000-01-05,6.494604,-1.153744,14.142953,0.216055,2000


In [37]:
climate['statefips'] = climate['fips'].apply(findState)

In [38]:
def celsius_to_fahrenheit(celsius):
    return (celsius * 9/5) + 32

In [39]:
climate['tMean'] = climate['tMean'].apply(celsius_to_fahrenheit)
climate['tMin'] = climate['tMin'].apply(celsius_to_fahrenheit)
climate['tMax'] = climate['tMax'].apply(celsius_to_fahrenheit)

In [40]:
import datetime

In [41]:
def convertTime(time):
    return datetime.strptime(time, '%Y-%m-%d')

In [42]:
def getMonth(date):
    return(date.strftime('%b'))

In [43]:
climate['date'] = pd.to_datetime(climate['date'])
climate['month'] = climate['date'].apply(getMonth)

In [44]:
climate['month'] = climate['month'].replace(month_replace)
climate.head()

,fips,date,tMean,tMin,tMax,prec,year,statefips,month
0,01001,2000-01-01,61.272425,50.160884,72.383967,0.560423,2000,01,1
1,01001,2000-01-02,63.342771,53.517596,73.167946,0.390251,2000,01,1
2,01001,2000-01-03,65.669476,57.201870,74.137082,0.216170,2000,01,1
3,01001,2000-01-04,61.166221,49.751427,72.581016,16.488676,2000,01,1
4,01001,2000-01-05,43.690288,29.923261,57.457315,0.216055,2000,01,1


In [45]:
climate = climate.merge(state[['statefips', 'state_name', 'state']], on = ['statefips'], how = 'inner')
climate['region'] = climate['state_name'].apply(get_region)
climate.head()

,fips,date,tMean,tMin,tMax,prec,year,statefips,month,state_name,state,region
0,01001,2000-01-01,61.272425,50.160884,72.383967,0.560423,2000,01,1,Alabama,AL,Southeast
1,01001,2000-01-02,63.342771,53.517596,73.167946,0.390251,2000,01,1,Alabama,AL,Southeast
2,01001,2000-01-03,65.669476,57.201870,74.137082,0.216170,2000,01,1,Alabama,AL,Southeast
3,01001,2000-01-04,61.166221,49.751427,72.581016,16.488676,2000,01,1,Alabama,AL,Southeast
4,01001,2000-01-05,43.690288,29.923261,57.457315,0.216055,2000,01,1,Alabama,AL,Southeast


In [46]:
climate = climate.drop(['fips', 'statefips', 'state_name', 'state', 'date'], axis = 1)
climate = climate.groupby(['year', 'month', 'region']).mean()
climate = climate.reset_index()
climate.head()

,year,month,region,tMean,tMin,tMax,prec
0,2000,1,Midwest,24.131735,13.734550,34.528919,1.101389
1,2000,1,Northeast,24.386417,14.782550,33.990284,2.720459
2,2000,1,Southeast,41.503673,30.796313,52.211033,3.218615
3,2000,1,Southwest,47.132753,33.657913,60.607594,1.114261
4,2000,1,West,30.973741,21.495705,40.451777,3.315519


In [47]:
suicide_climate = suicide_pop.merge(climate, on = ['year', 'month', 'region'], how = 'inner')
suicide_climate.head()

,year,region,month,deaths,pop,suicide_rate,tMean,tMin,tMax,prec
0,2000,Midwest,4,536,58609705,0.914524,48.604590,35.948717,61.260462,2.009247
1,2000,Midwest,8,583,58662683,0.993817,73.528029,61.980313,85.075744,2.368660
2,2000,Midwest,12,471,58715987,0.802167,15.491818,6.364439,24.619197,1.339481
3,2000,Midwest,2,498,58583129,0.850074,33.237830,22.512694,43.962965,1.778046
4,2000,Midwest,1,584,58569836,0.997100,24.131735,13.734550,34.528919,1.101389


In [48]:
summer_winter = [6, 7, 8, 12, 1, 2]
def is_summer_winter(month):
    if month in summer_winter :
        return 1
    else:
        return 0

In [49]:
suicide_climate['summer_winter'] = suicide_climate['month'].apply(is_summer_winter)
suicide_climate.head()

,year,region,month,deaths,pop,suicide_rate,tMean,tMin,tMax,prec,summer_winter
0,2000,Midwest,4,536,58609705,0.914524,48.604590,35.948717,61.260462,2.009247,0
1,2000,Midwest,8,583,58662683,0.993817,73.528029,61.980313,85.075744,2.368660,1
2,2000,Midwest,12,471,58715987,0.802167,15.491818,6.364439,24.619197,1.339481,1
3,2000,Midwest,2,498,58583129,0.850074,33.237830,22.512694,43.962965,1.778046,1
4,2000,Midwest,1,584,58569836,0.997100,24.131735,13.734550,34.528919,1.101389,1


In [50]:
suicide_climate.to_csv('monthly_GAM_region_all_groups.csv')